# Building an E-Commerce Clothing Classifier Model

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [2]:
# Load in the data
from torchvision import datasets
import torchvision.transforms as transforms

train_data = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transforms.ToTensor())
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

100.0%
100.0%
100.0%
100.0%


In [6]:
# View the results
train_data

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [7]:
test_data

Dataset FashionMNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [10]:
# View the number of classes
num_classes = len(train_data.classes)
# View the results
num_classes

10

In [11]:
# Set input channels and image size
num_input_channels=1
image_size= train_data[0][0].shape[1]

In [18]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNN, self).__init__()
        # Convolutional layer
        self.conv1 = nn.Conv2d(num_input_channels, 16, kernel_size=3, stride=1, padding=1)
        # ReLU activation function
        self.relu = nn.ReLU()
        # Max pooling layer
        self.maxpool = nn.MaxPool2d(2)
        # Fully connected layer
        self.fc = nn.Linear(16 * (image_size // 2) ** 2, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)  
        x = self.relu(x)   
        x = self.maxpool(x)
        x = x.view(x.size(0), -1) 
        x = self.fc(x)     
        return x

In [16]:
# Training the model
def train_model(net, num_epochs=1, batch_size=10):
    # Create data loader for training
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    # Loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(net.parameters(), lr=0.001)

    for epoch in range(num_epochs):
        running_loss = 0
        for features, labels in train_loader:
            optimizer.zero_grad()  # Zero gradients
            outputs = net(features)  # Forward pass
            loss = criterion(outputs, labels)  # Compute loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update weights
            running_loss += loss.item()  # Keep track of loss

        # Print loss for the current epoch
        print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}")

In [19]:
# Create the model and train it
model = SimpleCNN(num_classes)
train_model(model)

Epoch 1, Loss: 0.42092982680536806


In [20]:
# Testing the model
def test_model(net):
    # Create data loader for testing
    test_loader = DataLoader(test_data, batch_size=10, shuffle=False)

    # Initialize metrics
    accuracy_metric = Accuracy(task="multiclass", num_classes=num_classes)
    precision_metric = Precision(task="multiclass", num_classes=num_classes, average=None)
    recall_metric = Recall(task="multiclass", num_classes=num_classes, average=None)

    net.eval()  # Set the model to evaluation mode
    predictions = []

    # Iterate over test data and calculate metrics
    for features, labels in test_loader:
        outputs = net(features)
        preds = torch.argmax(outputs, dim=-1)
        predictions.extend(preds.tolist())
        
        # Update metrics
        accuracy_metric(preds, labels)
        precision_metric(preds, labels)
        recall_metric(preds, labels)

    # Print out the final metrics
    accuracy = accuracy_metric.compute().item()
    precision = precision_metric.compute().tolist()
    recall = recall_metric.compute().tolist()

    print(f"Accuracy: {accuracy}")
    print(f"Precision (per class): {precision}")
    print(f"Recall (per class): {recall}")

# Test the model after training
test_model(model)

Accuracy: 0.8787999749183655
Precision (per class): [0.7634042501449585, 0.9856410026550293, 0.7707581520080566, 0.899894654750824, 0.8303287625312805, 0.9652432799339294, 0.703661322593689, 0.937685489654541, 0.9875389337539673, 0.9577889442443848]
Recall (per class): [0.8970000147819519, 0.9610000252723694, 0.8539999723434448, 0.8539999723434448, 0.7829999923706055, 0.972000002861023, 0.6150000095367432, 0.9480000138282776, 0.9509999752044678, 0.953000009059906]
